In [10]:
import pandas as pd
import numpy as np
import setuptools
import openml
from sklearn.linear_model import LinearRegression 
import lightgbm as lgbm
import optuna
from sklearn.cluster import KMeans
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
from sklearn.preprocessing import StandardScaler
import gower
from sklearn_extra.cluster import KMedoids

In [2]:
#openml.config.apikey = 'FILL_IN_OPENML_API_KEY'  # set the OpenML Api Key
#SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite
print(benchmark_suite.tasks)

[361093, 361094, 361096, 361097, 361098, 361099, 361101, 361102, 361103, 361104, 361287, 361288, 361289, 361291, 361292, 361293, 361294]


In [16]:
# pip install pandas
# pip install setuptools
# pip install openml
# pip install lightgbm
# pip install optuna
# pip install engression


# numpy-1.26.3 pandas-2.1.4 setuptools-69.0.3 openml lightgbm-4.2.0  optuna-3.5.0

In [3]:
#openml.config.apikey = 'FILL_IN_OPENML_API_KEY'  # set the OpenML Api Key
#SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task = openml.tasks.get_task(361093)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [25]:
N_CLUSTERS=20

X_gower = X.copy()

for col in X_gower.select_dtypes(['category']).columns:
    X_gower[col] = X_gower[col].astype('object')

gower_dist_matrix = gower.gower_matrix(X_gower)

kmedoids = KMedoids(n_clusters=N_CLUSTERS, random_state=0, metric='precomputed').fit(gower_dist_matrix)
distances=[]
gower_dist=[]
counts=[]
ideal_len=len(kmedoids.labels_)/5

for i in np.arange(N_CLUSTERS):
    cluster_data = X_gower.loc[kmedoids.labels_==i,:]
    # Compute the Gower distance between each data point in the cluster and each data point in the global dataset
    distances_matrix = gower.gower_matrix(cluster_data, X_gower)
    # Compute the average distance
    average_distance = np.mean(distances_matrix)
    gower_dist.append(average_distance)
    counts.append(cluster_data.shape[0])

dist_df=pd.DataFrame(data={'gower_dist': gower_dist, 'count': counts}, index=np.arange(N_CLUSTERS))
dist_df=dist_df.sort_values('gower_dist', ascending=False)
dist_df['cumulative_count']=dist_df['count'].cumsum()
dist_df['abs_diff']=np.abs(dist_df['cumulative_count']-ideal_len)

final=(np.where(dist_df['abs_diff']==np.min(dist_df['abs_diff']))[0])[0]
labelss=dist_df.index[0:final+1].to_list()
labels=pd.Series(kmedoids.labels_).isin(labelss)
labels.index=X.index
close_index=labels.index[np.where(labels==False)[0]]
far_index=labels.index[np.where(labels==True)[0]]

X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]


X_gower_ = X_train.copy()

for col in X_gower_.select_dtypes(['category']).columns:
    X_gower_[col] = X_gower_[col].astype('object')

gower_dist_matrix_ = gower.gower_matrix(X_gower_)

kmedoids_ = KMedoids(n_clusters=N_CLUSTERS, random_state=0, metric='precomputed').fit(gower_dist_matrix_)
distances_=[]
gower_dist_=[]
counts_=[]
ideal_len_=len(kmedoids.labels_)/5

for i in np.arange(N_CLUSTERS):
    cluster_data_ = X_gower_.loc[kmedoids_.labels_==i,:]
    # Compute the Gower distance between each data point in the cluster and each data point in the global dataset
    distances_matrix_ = gower.gower_matrix(cluster_data_, X_gower_)
    # Compute the average distance
    average_distance_ = np.mean(distances_matrix_)
    gower_dist_.append(average_distance_)
    counts_.append(cluster_data_.shape[0])

dist_df_=pd.DataFrame(data={'gower_dist': gower_dist_, 'count': counts_}, index=np.arange(N_CLUSTERS))
dist_df_=dist_df_.sort_values('gower_dist', ascending=False)
dist_df_['cumulative_count']=dist_df_['count'].cumsum()
dist_df_['abs_diff']=np.abs(dist_df_['cumulative_count']-ideal_len_)

final_=(np.where(dist_df_['abs_diff']==np.min(dist_df_['abs_diff']))[0])[0]
labelss_=dist_df_.index[0:final_+1].to_list()
labels_=pd.Series(kmedoids_.labels_).isin(labelss_)
labels_.index=X_train.index
close_index_=labels_.index[np.where(labels_==False)[0]]
far_index_=labels_.index[np.where(labels_==True)[0]]

X_train_ = X_train.loc[close_index_,:]
X_val = X_train.loc[far_index_,:]
y_train_ = y_train.loc[close_index_]
y_val = y_train.loc[far_index_]

c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn_extra\cluster\_k_medoids.py:329: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(
c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn_extra\cluster\_k_medoids.py:329: UserWarning: Cluster 3 is empty! self.labels_[self.medoid_indices_[3]] may not be labeled with its corresponding cluster (3).
  warnings.warn(
c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn_extra\cluster\_k_medoids.py:329: UserWarning: Cluster 4 is empty! self.labels_[self.medoid_indices_[4]] may not be labeled with its corresponding cluster (4).
  warnings.warn(
c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn_extra\cluster\_k_medoids.py:329: UserWarning: Cluster 5 is empty! self.labels_[self.medoid_indices_[5]] may not be labeled with its corresponding cl

In [39]:
N_TRIALS=5

def gp(trial):

    params = {'lenghtscale': trial.suggest_float('lenghtscale', 0, 10)}
    
    gp=GaussianProcessRegressor(kernel=Matern(length_scale=params['lenghtscale'], nu=1.5))
    gp.fit(X_train_, y_train_)
    y_val_hat_gp=gp.predict(X_val)
    RMSE_gp=np.sqrt(np.mean((y_val-y_val_hat_gp)**2))
    return RMSE_gp

sampler_gp = optuna.samplers.TPESampler(seed=10)
study_gp = optuna.create_study(sampler=sampler_gp, direction='minimize')
study_gp.optimize(gp, n_trials=N_TRIALS)

gp_model=GaussianProcessRegressor(kernel=Matern(length_scale=study_gp.best_params['lenghtscale'], nu=1.5))


def boosted(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
              'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              'min_child_samples': trial.suggest_int('min_child_samples', 10, 100)}
    
    boosted_tree_model=lgbm.LGBMRegressor(**params)
    boosted_tree_model.fit(X_train_, y_train_)
    y_val_hat_boost=boosted_tree_model.predict(X_val)
    RMSE_boost=np.sqrt(np.mean((y_val-y_val_hat_boost)**2))

    return RMSE_boost

sampler_boost = optuna.samplers.TPESampler(seed=10)
study_boost = optuna.create_study(sampler=sampler_boost, direction='minimize')
study_boost.optimize(boosted, n_trials=N_TRIALS)

boosted_model=lgbm.LGBMRegressor(**study_boost.best_params)

def rf(trial):

    params = {'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              'max_features': trial.suggest_int('max_features', 1, 30),
              'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 100)}
    
    rf_model=RandomForestRegressor(**params)
    rf_model.fit(X_train_, y_train_)
    y_val_hat_rf=rf_model.predict(X_val)
    RMSE_rf=np.sqrt(np.mean((y_val-y_val_hat_rf)**2))

    return RMSE_rf

sampler_rf = optuna.samplers.TPESampler(seed=10)
study_rf = optuna.create_study(sampler=sampler_rf, direction='minimize')
study_rf.optimize(rf, n_trials=N_TRIALS)

rf_model=RandomForestRegressor(**study_rf.best_params)


def engressor_NN(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
              'num_epoches': trial.suggest_int('num_epoches', 100, 1000),
              'num_layer': trial.suggest_int('num_layer', 2, 5),
              'hidden_dim': trial.suggest_int('hidden_dim', 50, 100),
              'noise_dim': trial.suggest_int('noise_dim', 50, 100),}
    
    engressor_model=engression(torch.Tensor(np.array(X_train_)), torch.Tensor(np.array(y_train_).reshape(-1,1)), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
    y_val_hat_engression=engressor_model.predict(torch.Tensor(np.array(X_val)), target="mean")
    RMSE_engression=np.sqrt((((torch.Tensor(np.array(y_val).reshape(-1,1)))-y_val_hat_engression)**2).mean(axis=0))

    return RMSE_engression

sampler_engression = optuna.samplers.TPESampler(seed=10)
study_engression = optuna.create_study(sampler=sampler_engression, direction='minimize')
study_engression.optimize(engressor_NN, n_trials=N_TRIALS)

[I 2023-12-21 10:52:51,669] A new study created in memory with name: no-name-e98de119-9afe-4473-a4e8-08087d0430c4
[I 2023-12-21 10:56:12,684] Trial 0 finished with value: 0.22163665540145625 and parameters: {'lenghtscale': 7.71320643266746}. Best is trial 0 with value: 0.22163665540145625.
[I 2023-12-21 10:56:40,116] Trial 1 finished with value: 0.22163665540145625 and parameters: {'lenghtscale': 0.207519493594015}. Best is trial 0 with value: 0.22163665540145625.
[I 2023-12-21 11:00:47,139] Trial 2 finished with value: 0.22163665540145625 and parameters: {'lenghtscale': 6.336482349262754}. Best is trial 0 with value: 0.22163665540145625.
[I 2023-12-21 11:10:06,626] Trial 3 finished with value: 0.22163665540145625 and parameters: {'lenghtscale': 7.488038825386118}. Best is trial 0 with value: 0.22163665540145625.
[I 2023-12-21 11:13:37,530] Trial 4 finished with value: 0.22163665540145625 and parameters: {'lenghtscale': 4.9850701230259045}. Best is trial 0 with value: 0.221636655401456

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000888 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4650
[LightGBM] [Info] Number of data points in the train set: 6454, number of used features: 21
[LightGBM] [Info] Start training from score 87.332042


[I 2023-12-21 11:13:38,042] Trial 0 finished with value: 39.98724469883315 and parameters: {'learning_rate': 0.38588900099010626, 'n_estimators': 108, 'reg_lambda': 0.005044685709888605, 'max_depth': 23, 'min_child_samples': 55}. Best is trial 0 with value: 39.98724469883315.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001422 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4650
[LightGBM] [Info] Number of data points in the train set: 6454, number of used features: 21
[LightGBM] [Info] Start training from score 87.332042
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

[I 2023-12-21 11:13:38,286] Trial 1 finished with value: 25.73056761777012 and parameters: {'learning_rate': 0.11317352611989298, 'n_estimators': 179, 'reg_lambda': 0.0699481785242808, 'max_depth': 6, 'min_child_samples': 18}. Best is trial 1 with value: 25.73056761777012.


...


Training loss on the original (non-standardized) scale:
	Energy-loss: 1.5929,  E(|Y-Yhat|): 2.9787,  E(|Yhat-Yhat'|): 2.7716

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 
Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 7] energy-loss: 0.7053,  E(|Y-Yhat|): 1.5619,  E(|Yhat-Yhat'|): 1.7132
[Epoch 100 (13%), batch 7] energy-loss: 0.1366,  E(|Y-Yhat|): 0.2675,  E(|Yhat-Yhat'|): 0.2620
[Epoch 200 (27%), batch 7] energy-loss: 0.1310,  E(|Y-Yhat|): 0.2856,  E(|Yhat-Yhat'|): 0.3092
[Epoch 300 (40%), batch 7] energy-loss: 0.1431,  E(|Y-Yhat|): 0.2960,  E(|Yhat-Yhat'|): 0.3057
[Epoch 400 (53%)

[I 2023-12-21 11:22:42,722] Trial 3 finished with value: 265.4168395996094 and parameters: {'learning_rate': 0.3066505073478647, 'num_epoches': 750, 'num_layer': 3, 'hidden_dim': 96, 'noise_dim': 86}. Best is trial 0 with value: 68.07936096191406.



Training loss on the original (non-standardized) scale:
	Energy-loss: 1.2768,  E(|Y-Yhat|): 2.5015,  E(|Yhat-Yhat'|): 2.4494

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 
Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 7] energy-loss: 0.7314,  E(|Y-Yhat|): 0.9723,  E(|Yhat-Yhat'|): 0.4817
[Epoch 100 (43%), batch 7] energy-loss: 0.1408,  E(|Y-Yhat|): 0.2796,  E(|Yhat-Yhat'|): 0.2776
[Epoch 200 (87%), batch 7] energy-loss: 0.1703,  E(|Y-Yhat|): 0.3132,  E(|Yhat-Yhat'|): 0.2858


[I 2023-12-21 11:23:50,886] Trial 4 finished with value: 77.3936996459961 and parameters: {'learning_rate': 0.27172963963761937, 'num_epoches': 228, 'num_layer': 3, 'hidden_dim': 84, 'noise_dim': 72}. Best is trial 0 with value: 68.07936096191406.



Training loss on the original (non-standardized) scale:
	Energy-loss: 1.4006,  E(|Y-Yhat|): 2.9966,  E(|Yhat-Yhat'|): 3.1921

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 


In [40]:
gp_model.fit(X_train, y_train)
y_test_hat_gp=gp_model.predict(X_test)
RMSE_gp=np.sqrt(np.mean((y_test-y_test_hat_gp)**2))

boosted_model.fit(X_train, y_train)
y_test_hat_boosted=boosted_model.predict(X_test)
RMSE_boosted=np.sqrt(np.mean((y_test-y_test_hat_boosted)**2))

rf_model.fit(X_train, y_train)
y_test_hat_rf=rf_model.predict(X_test)
RMSE_rf=np.sqrt(np.mean((y_test-y_test_hat_rf)**2))

lin_reg=LinearRegression()
lin_reg.fit(X_train, y_train)
y_test_hat_linreg=lin_reg.predict(X_test)
RMSE_linreg=np.sqrt(np.mean((y_test-y_test_hat_linreg)**2))

params=study_engression.best_params
engressor_model=engression(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(y_train).reshape(-1,1)), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
y_test_hat_engression=engressor_model.predict(torch.Tensor(np.array(X_test)), target="mean")
RMSE_engression=np.sqrt((((torch.Tensor(np.array(y_test).reshape(-1,1)))-y_test_hat_engression)**2).mean(axis=0))

print("RMSE linear regression: ",RMSE_linreg)
print("RMSE boosted trees", RMSE_boosted)
print("RMSE random forest", RMSE_rf)
print("RMSE gaussian process", RMSE_gp)
print("RMSE engression", RMSE_engression)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003752 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4860
[LightGBM] [Info] Number of data points in the train set: 6739, number of used features: 21
[LightGBM] [Info] Start training from score 83.640451
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

In [43]:
from MODELS import ft_transformer

ModuleNotFoundError: No module named 'models'

In [44]:
pip install pytorch-widedeep

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 45.9/45.9 kB 450.9 kB/s eta 0:00:00
     --------------------------------------- 65.6/65.6 kB 66.9 kB/s eta 0:00:00
     ------------------------------------ 181.6/181.6 kB 219.5 kB/s eta 0:00:00
   -                                        0.6/21.9 MB ? eta -:--:--
   -------------------------------------- 667.9/667.9 kB 132.8 kB/s eta 0:00:00
   ---------------------------------------- 1.1/1.1 MB 8.1 MB/s eta 0:00:00
   ---------------------------------------- 192.3/192.3 MB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 44.6/44.6 kB ? eta 0:00:00
   ---------------------------------------- 24.0/24.0 MB 7.3 MB/s eta 0:00:00
   ---------------------------------------- 12.1/12.1 MB 8.2 MB/s eta 0:00:00
   ---------------------------------------- 806.1/806.1 kB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 1

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    unknown package:
        Expected sha256 4f7e9edc893972570fb04f87c8ce25724ad9f0aea2ef9cc183c1eb1cd6f24715
             Got        72c2cb405f6031a3cc0ed7b3e65583d94fccbb22812e3af34b1398a0f7c9c986


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
pip install rpy2

Note: you may need to restart the kernel to use updated packages.Collecting rpy2
     ------------------------------------- 219.8/219.8 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for rpy2: filename=rpy2-3.5.15-py3-none-any.whl size=220723 sha256=530ca46a0dd7324db278ba1e13cc78f56c704071e82db30635b160721bcf8668
  Stored in directory: c:\users\dalma\appdata\local\pip\cache\wheels\af\93\69\c4904a387a34629ae40162ce1e61479cdd4ae3fd40078a9db3
Successfully built rpy2




[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Install and load the ddalpha package
utils = importr('utils')
utils.install_packages('ddalpha')
ddalpha = importr('ddalpha')

--- Per piacere, seleziona un mirror CRAN per la sessione ---


Exception ignored from cffi callback <function _consolewrite_ex at 0x000001F4174783A0>:
Traceback (most recent call last):
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 78: invalid continuation byte






package 'ddalpha' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\dalma\AppData\Local\Temp\Rtmpy0n8on\downloaded_packages


In [50]:
# Convert pandas DataFrame to R matrix
r_dataset = robjects.r['matrix'](robjects.FloatVector(X.values.flatten()), nrow=X.shape[0], byrow=True)

# Compute the center of the dataset
center = robjects.r['colMeans'](r_dataset)

# Function to compute simplicial distance for a single point
def compute_simplicial_distance(data_point):
    return ddalpha.dd_alpha(data_point, center)

# Compute simplicial distance for each data point
simplicial_distances = [compute_simplicial_distance(robjects.FloatVector(row)) for _, row in X.iterrows()]

AttributeError: module 'ddalpha' has no attribute 'dd_alpha'

In [51]:
# Convert pandas DataFrame to R matrix
r_dataset = robjects.r['matrix'](robjects.FloatVector(X.values.flatten()), nrow=X.shape[0], byrow=True)

# Compute the center of the dataset
center = robjects.r['colMeans'](r_dataset)

# Function to compute simplicial distance for a single point
def compute_simplicial_distance(data_point):
    return ddalpha.dd_alpha_div(data_point, center)[0]

# Compute simplicial distances for each data point
simplicial_distances = X.apply(lambda row: compute_simplicial_distance(robjects.FloatVector(row)), axis=1)

# Create a pandas Series with the same index as X
simplicial_distances_series = pd.Series(simplicial_distances, index=X.index, name='SimplicialDistance')

AttributeError: module 'ddalpha' has no attribute 'dd_alpha_div'

In [47]:
X

,lread,lwrite,scall,sread,swrite,fork,exec,rchar,wchar,pgout,...,pgfree,pgscan,atch,pgin,ppgin,pflt,vflt,runqsz,freemem,freeswap
0,6.0,2.0,1036.0,103.0,114.0,1.00,1.00,172076.0,355965.0,0.00,...,0.00,0.00,0.00,2.00,4.00,73.60,89.00,2.0,6527.0,1851864.0
1,1.0,0.0,2165.0,205.0,101.0,0.40,1.20,43107.0,44139.0,4.80,...,75.80,181.40,0.20,85.40,88.20,19.40,161.80,3.0,130.0,1131931.0
2,62.0,77.0,3806.0,258.0,166.0,1.40,1.40,492142.0,268706.0,4.80,...,44.00,79.20,2.20,7.60,12.20,68.00,218.80,5.2,256.0,1314590.0
3,5.0,0.0,4721.0,256.0,177.0,0.99,2.58,524787.0,174964.0,14.51,...,88.47,189.86,1.99,4.17,24.85,95.63,248.91,1.0,233.0,972606.0
4,42.0,55.0,3949.0,249.0,244.0,2.60,4.60,197289.0,529200.0,4.20,...,6.60,0.00,1.40,1.80,2.20,219.60,297.20,3.4,331.0,1013805.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,74.0,49.0,2688.0,176.0,103.0,11.00,32.20,57714.0,38484.0,0.80,...,1.00,0.00,0.00,0.80,0.80,343.20,649.40,7.0,314.0,1096333.0
8188,29.0,40.0,1906.0,118.0,90.0,0.80,2.00,8175.0,27313.0,0.00,...,0.00,0.00,0.00,0.80,0.80,56.20,78.60,3.6,166.0,1107088.0
8189,3.0,0.0,926.0,90.0,67.0,0.60,1.00,5411.0,19322.0,0.00,...,0.00,0.00,0.40,0.40,0.40,53.40,154.00,1.0,1177.0,1020400.0
8190,4.0,0.0,418.0,30.0,29.0,0.80,1.00,3959.0,10679.0,0.00,...,0.00,0.00,0.00,0.20,0.20,61.00,73.20,2.4,6355.0,1702592.0


In [54]:
# Convert pandas DataFrame to R matrix
r_dataset = robjects.r['matrix'](robjects.FloatVector(X.values.flatten()), nrow=X.shape[0], byrow=True)

# Compute the center of the dataset
center = robjects.r['colMeans'](r_dataset)

# Function to compute simplicial distance for a single point
def compute_simplicial_distance(data_point):
    return ddalpha.depth.simplicial(data_point, center)[0]

# Compute simplicial distances for each data point
simplicial_distances = X.apply(lambda row: compute_simplicial_distance(robjects.FloatVector(row)), axis=1)

# Create a pandas Series with the same index as X
simplicial_distances_series = pd.Series(simplicial_distances, index=X.index, name='SimplicialDistance')

AttributeError: module 'ddalpha' has no attribute 'depth'

In [65]:
import pandas as pd
from rpy2.robjects import r, pandas2ri

#pandas2ri.activate()
#r_dataframe = pandas2ri.pydataframe_to_rdataframe(X)

# Compute the center of the dataset
# center = r("colMeans")(r_dataframe)


# Convert pandas DataFrame to R matrix
r_dataset = robjects.r['matrix'](robjects.FloatVector(X.values.flatten()), nrow=X.shape[0], byrow=True)

# Compute the center of the dataset
center = robjects.r['colMeans'](r_dataset)

# Function to compute simplicial distance for a single point
def compute_simplicial_distance(data_point):
    return r("ddalpha.depth.simplicial")(data_point, center)[0]

# Compute simplicial distances for each data point
simplicial_distances = X.apply(lambda row: compute_simplicial_distance(row), axis=1)

# Create a pandas Series with the same index as X
simplicial_distances_series = pd.Series(simplicial_distances, index=X.index, name='SimplicialDistance')

  object 'ddalpha.depth.simplicial' not found



RRuntimeError: Error in (function (expr, envir = parent.frame(), enclos = if (is.list(envir) ||  : 
  object 'ddalpha.depth.simplicial' not found


In [58]:
pandas2ri.activate()
r_dataframe = pandas2ri.py2ri(X)

AttributeError: module 'rpy2.robjects.pandas2ri' has no attribute 'py2ri'

In [59]:
pip install gudhi


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 2.7/2.7 MB 6.5 MB/s eta 0:00:00


In [64]:
import subprocess
import pandas as pd

df = X

# Path to the R script
r_script_path = "compute_simplicial_distance.R"

# Placeholder for the results
simplicial_distances = []

# Iterate through rows of the DataFrame and call the R script
for _, row in df.iterrows():
    command = ["Rscript", r_script_path] + list(map(str, row.values)) + list(map(str, df.mean().values))
    result = subprocess.check_output(command, universal_newlines=True)
    simplicial_distances.append(float(result.strip()))

# Create a pandas Series with the same index as df
simplicial_distances_series = pd.Series(simplicial_distances, index=df.index, name='SimplicialDistance')

# Add the new column to the DataFrame
df = pd.concat([df, simplicial_distances_series], axis=1)

# Print the DataFrame with the new column
print(df)

CalledProcessError: Command '['Rscript', 'compute_simplicial_distance.R', '6.0', '2.0', '1036.0', '103.0', '114.0', '1.0', '1.0', '172076.0', '355965.0', '0.0', '0.0', '0.0', '0.0', '0.0', '2.0', '4.0', '73.6', '89.0', '2.0', '6527.0', '1851864.0', '19.5596923828125', '13.106201171875', '2306.3182373046875', '210.47998046875', '150.0582275390625', '1.884554443359375', '2.791998291015625', '197013.66833496094', '95898.29064941406', '2.2853173828125', '5.97722900390625', '11.9197119140625', '21.526849365234376', '1.1275048828125', '8.277960205078125', '12.388586425781249', '109.79379882812499', '185.3157958984375', '19.63067626953125', '1763.456298828125', '1328125.9598388672']' returned non-zero exit status 1.

In [66]:
import pandas as pd
from rpy2 import robjects
from rpy2.robjects import pandas2ri

# Install and load the ddalpha package
robjects.r("install.packages('ddalpha')")
robjects.r("library(ddalpha)")

Exception ignored from cffi callback <function _consolewrite_ex at 0x000001F4174783A0>:
Traceback (most recent call last):
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 24: invalid continuation byte


In [67]:
import pandas as pd
from rpy2 import robjects
from rpy2.robjects import pandas2ri

# Function to install R packages with error handling
def install_r_package(package_name):
    try:
        robjects.r(f"install.packages('{package_name}', dependencies=TRUE)")
    except Exception as e:
        print(f"Error installing R package {package_name}: {e}")

# Install and load the ddalpha package
install_r_package('ddalpha')
robjects.r("library(ddalpha)")

Exception ignored from cffi callback <function _consolewrite_ex at 0x000001F4174783A0>:
Traceback (most recent call last):
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 24: invalid continuation byte


In [80]:
import pandas as pd
from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# Install and load the ddalpha package
utils = importr('utils')
utils.install_packages('ddalpha')
ddalpha = importr('ddalpha')

# Example DataFrame
'''data = {'feature1': [0.1, 0.2, 0.5, 0.4],
        'feature2': [0.2, 0.3, 0.3, 0.2],
        'feature3': [0.3, 0.4, 0.1, 0.2],
        'feature4': [0.4, 0.1, 0.1, 0.2]}

X = pd.DataFrame(data)'''

# Check for missing values in the DataFrame
if X.isnull().values.any():
    raise ValueError("DataFrame contains missing values. Please handle or remove them.")

# Convert pandas DataFrame to R data frame
pandas2ri.activate()
r_dataframe = pandas2ri.py2rpy(X)

# Compute the center of the dataset
center = robjects.r["colMeans"](r_dataframe)

# Function to compute simplicial distance for a single point
def compute_simplicial_distance(data_point):
    return ddalpha.depth_simplicial(data_point, r_dataframe)[0]

# Compute simplicial distances for each data point
simplicial_distances = X.apply(lambda row: compute_simplicial_distance(robjects.FloatVector(row)), axis=1)

# Create a pandas Series with the same index as X
simplicial_distances_series = pd.Series(simplicial_distances, index=X.index, name='SimplicialDistance')

# Print or use the resulting Series as needed
print(simplicial_distances_series)


Exception ignored from cffi callback <function _consolewrite_ex at 0x000001F4174783A0>:
Traceback (most recent call last):
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 24: invalid continuation byte
  NAs in foreign function call (arg 8)





RRuntimeError: Error in (function (x, data, exact = F, k = 0.05, seed = 0)  : 
  NAs in foreign function call (arg 8)


In [95]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

# Install and load the ddalpha package
utils = importr('utils')
utils.install_packages('ddalpha')
ddalpha = importr('ddalpha')

# Convert pandas DataFrame to R data frame using numpy2ri
with (ro.default_converter + pandas2ri.converter).context():
  r_dataframe = ro.conversion.get_conversion().py2rpy(X)

# Ensure that data is a numeric matrix
ro.r.assign("data_matrix", r_dataframe)
ro.r("data_matrix <- as.matrix(data_matrix)")

# Compute the center of the dataset
center = ro.r["colMeans"](ro.r("data_matrix"))

# Function to compute simplicial distance for a single point
def compute_simplicial_distance(data_point):
    return ddalpha.depth_simplicial(ro.r("matrix")(data_point, ncol=len(data_point)), r_dataframe, exact=False, k=0.05, seed=0)[0]

# Compute simplicial distances for each data point
simplicial_distances = X.apply(lambda row: compute_simplicial_distance(row), axis=1)

# Create a pandas Series with the same index as X
simplicial_distances_series = pd.Series(simplicial_distances, index=X.index, name='SimplicialDistance')

# Print or use the resulting Series as needed
print(simplicial_distances_series)

Exception ignored from cffi callback <function _consolewrite_ex at 0x000001F4174783A0>:
Traceback (most recent call last):
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 24: invalid continuation byte
  NAs in foreign function call (arg 8)





RRuntimeError: Error in (function (x, data, exact = F, k = 0.05, seed = 0)  : 
  NAs in foreign function call (arg 8)


In [96]:
np.exp(0.5)+np.exp(1/3)

3.044333695786218

In [100]:
60*20/(60+85)

8.275862068965518

In [90]:
ddalpha.depth_simplicial(r_dataframe, r_dataframe)[0]

  NAs in foreign function call (arg 8)





RRuntimeError: Error in (function (x, data, exact = F, k = 0.05, seed = 0)  : 
  NAs in foreign function call (arg 8)


In [89]:
r_dataframe

lread,lwrite,scall,...,runqsz,freemem,freeswap
...,...,...,...,...,...,...


In [75]:
center

array([0.3 , 0.25, 0.25, 0.2 ])

In [82]:
np.isnan(X).sum(axis=0)

lread       0
lwrite      0
scall       0
sread       0
swrite      0
fork        0
exec        0
rchar       0
wchar       0
pgout       0
ppgout      0
pgfree      0
pgscan      0
atch        0
pgin        0
ppgin       0
pflt        0
vflt        0
runqsz      0
freemem     0
freeswap    0
dtype: int64

In [5]:
import pandas as pd
import subprocess
from rpy2.robjects import pandas2ri, r

In [6]:
df=X

In [10]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
with (ro.default_converter + pandas2ri.converter).context():
  r_dataframe = ro.conversion.get_conversion().py2rpy(df)

In [11]:
r_dataframe

lread,lwrite,scall,...,runqsz,freemem,freeswap
6.000000,2.000000,1036.000000,...,2.000000,6527.000000,1851864.000000
1.000000,0.000000,2165.000000,,3.000000,130.000000,1131931.000000
62.000000,77.000000,3806.000000,,5.200000,256.000000,1314590.000000
5.000000,0.000000,4721.000000,,1.000000,233.000000,972606.000000
...,...,...,,...,...,...
29.000000,40.000000,1906.000000,,3.600000,166.000000,1107088.000000
3.000000,0.000000,926.000000,,1.000000,1177.000000,1020400.000000
4.000000,0.000000,418.000000,,2.400000,6355.000000,1702592.000000
5.000000,0.000000,1888.000000,,4.600000,1628.000000,1757696.000000


In [13]:
r.assign('my_data', r_dataframe)

lread,lwrite,scall,...,runqsz,freemem,freeswap
6.000000,2.000000,1036.000000,...,2.000000,6527.000000,1851864.000000
1.000000,0.000000,2165.000000,,3.000000,130.000000,1131931.000000
62.000000,77.000000,3806.000000,,5.200000,256.000000,1314590.000000
5.000000,0.000000,4721.000000,,1.000000,233.000000,972606.000000
...,...,...,,...,...,...
29.000000,40.000000,1906.000000,,3.600000,166.000000,1107088.000000
3.000000,0.000000,926.000000,,1.000000,1177.000000,1020400.000000
4.000000,0.000000,418.000000,,2.400000,6355.000000,1702592.000000
5.000000,0.000000,1888.000000,,4.600000,1628.000000,1757696.000000


In [15]:
# Save the R script to a file
r_script = """
if (!requireNamespace("ddalpha", quietly = TRUE)) {
  install.packages("ddalpha")
}
library(ddalpha)

# Load your DataFrame from the Python environment
my_data <- my_data  # Make sure to use the same name you assigned in Python

# Compute the simplicial depth with respect to the centroid
simplicial_depth <- depth.simplicial(my_data)

# Print the result
print(simplicial_depth)
"""

with open('script.R', 'w') as f:
    f.write(r_script)

In [16]:
# Run the R script from Python
subprocess.run(['Rscript', 'script.R'], check=True)

CalledProcessError: Command '['Rscript', 'script.R']' returned non-zero exit status 1.

In [27]:
X

,lread,lwrite,scall,sread,swrite,fork,exec,rchar,wchar,pgout,...,pgfree,pgscan,atch,pgin,ppgin,pflt,vflt,runqsz,freemem,freeswap
0,6.0,2.0,1036.0,103.0,114.0,1.00,1.00,172076.0,355965.0,0.00,...,0.00,0.00,0.00,2.00,4.00,73.60,89.00,2.0,6527.0,1851864.0
1,1.0,0.0,2165.0,205.0,101.0,0.40,1.20,43107.0,44139.0,4.80,...,75.80,181.40,0.20,85.40,88.20,19.40,161.80,3.0,130.0,1131931.0
2,62.0,77.0,3806.0,258.0,166.0,1.40,1.40,492142.0,268706.0,4.80,...,44.00,79.20,2.20,7.60,12.20,68.00,218.80,5.2,256.0,1314590.0
3,5.0,0.0,4721.0,256.0,177.0,0.99,2.58,524787.0,174964.0,14.51,...,88.47,189.86,1.99,4.17,24.85,95.63,248.91,1.0,233.0,972606.0
4,42.0,55.0,3949.0,249.0,244.0,2.60,4.60,197289.0,529200.0,4.20,...,6.60,0.00,1.40,1.80,2.20,219.60,297.20,3.4,331.0,1013805.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,74.0,49.0,2688.0,176.0,103.0,11.00,32.20,57714.0,38484.0,0.80,...,1.00,0.00,0.00,0.80,0.80,343.20,649.40,7.0,314.0,1096333.0
8188,29.0,40.0,1906.0,118.0,90.0,0.80,2.00,8175.0,27313.0,0.00,...,0.00,0.00,0.00,0.80,0.80,56.20,78.60,3.6,166.0,1107088.0
8189,3.0,0.0,926.0,90.0,67.0,0.60,1.00,5411.0,19322.0,0.00,...,0.00,0.00,0.40,0.40,0.40,53.40,154.00,1.0,1177.0,1020400.0
8190,4.0,0.0,418.0,30.0,29.0,0.80,1.00,3959.0,10679.0,0.00,...,0.00,0.00,0.00,0.20,0.20,61.00,73.20,2.4,6355.0,1702592.0


In [29]:
r.assign('r_dataframe', r_dataframe)

lread,lwrite,scall,...,runqsz,freemem,freeswap
6.000000,2.000000,1036.000000,...,2.000000,6527.000000,1851864.000000
1.000000,0.000000,2165.000000,,3.000000,130.000000,1131931.000000
62.000000,77.000000,3806.000000,,5.200000,256.000000,1314590.000000
5.000000,0.000000,4721.000000,,1.000000,233.000000,972606.000000
...,...,...,,...,...,...
29.000000,40.000000,1906.000000,,3.600000,166.000000,1107088.000000
3.000000,0.000000,926.000000,,1.000000,1177.000000,1020400.000000
4.000000,0.000000,418.000000,,2.400000,6355.000000,1702592.000000
5.000000,0.000000,1888.000000,,4.600000,1628.000000,1757696.000000


In [31]:
import rpy2.robjects as robjects
robjects.r.source("compute_simplicial_distance.r", encoding="utf-8")

  NAs in foreign function call (arg 8)





RRuntimeError: Error in depth.simplicial(r_dataframe, r_dataframe) : 
  NAs in foreign function call (arg 8)


In [24]:
subprocess.call (['Rscript', 'compute_simplicial_distance.r'])

1

In [22]:
# Retrieve the result from R
result_in_python = r['simplicial_depth']

# Convert the result to a Pandas Series
result_series = pandas2ri.ri2py(result_in_python)

KeyError: "'simplicial_depth' not found"

In [18]:
# Run the R script from Python
subprocess.run(['compute_simplicial_distance.r'], check=True)

OSError: [WinError 193] %1 non è un'applicazione di Win32 valida